Я использовал антивирус Alphamountain для категоризации содержимого файла new_url_task_2.csv. Этот скрипт позволяет отправлять запросы к API для получения информации о категориях и угрозах для URL. Для использования потребуется лицензия, которую нужно запрашивать у владельцев сервиса, что замедляет процесс. В коде предусмотрены функции для получения категории и угроз для каждого URL, а также обработка прогресса с выводом результатов в CSV файл. Каждые 10 обработанных URL сохраняются в отдельный файл, а результаты по завершении обработки всех URL сохраняются в итоговый файл.

In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time

# Загрузите данные из CSV
df = pd.read_csv('new_url_task_2.csv')

# Убедитесь, что есть только 1000 URL
df = df.iloc[0001:1000]


# Лицензионный ключ
license_key = ''

# Функция для запроса категории URL
def get_category(url):
    url = url if url.startswith("http") else "https://" + url  # добавляем протокол, если его нет
    payload = {
        "uri": url,
        "license": license_key,
        "type": "partner.info",
        "version": 1
    }
    try:
        response = requests.post("https://api.alphamountain.ai/category/uri/", json=payload, timeout=10)
        if response.status_code == 200:
            return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при получении категории для {url}: {e}")
    return None

# Функция для запроса угроз URL
def get_threat(url):
    url = url if url.startswith("http") else "https://" + url  # добавляем протокол, если его нет
    payload = {
        "uri": url,
        "license": license_key,
        "type": "partner.info",
        "version": 1
    }
    try:
        response = requests.post("https://api.alphamountain.ai/threat/uri/", json=payload, timeout=10)
        if response.status_code == 200:
            return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при получении угрозы для {url}: {e}")
    return None

# Создание DataFrame для записи результатов
results_df = pd.DataFrame(columns=['url', 'Category', 'Threat'])

# Прогресс-бар и выполнение запросов
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Категоризация URL"):
    url = row['url']
    
    # Получение категории и угрозы
    category = get_category(url)
    threat = get_threat(url)
    
    # Добавляем в таблицу результатов
    results_df = pd.concat([results_df, pd.DataFrame({'url': [url], 'Category': [category], 'Threat': [threat]})], ignore_index=True)
    
    # Сохраняем результаты в файл каждые 10 URL
    if index % 10 == 0:
        results_df.to_csv('categorized_results_partial_2.csv', index=False)

    # Краткая пауза для предотвращения перегрузки API
    time.sleep(1)

# Сохранение финального результата
results_df.to_csv('categorized_new_url_task_2_final_2.csv', index=False)
print("Категоризация завершена!")


Категоризация URL: 100%|██████████| 999/999 [44:30<00:00,  2.67s/it]

Категоризация завершена!
